In [1]:
import numpy as np
import pandas as pd
import os, tempfile
os.chdir(tempfile.mkdtemp())
pd.DataFrame(np.random.random((987,10)),columns=[f'c{i}' for i in range(10)]).to_csv('data.csv',index=False)
!ls -hl
!head -2 data.csv

total 188K
-rw-rw-r-- 1 hoi hoi 186K Jul 27 11:21 data.csv
c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0.22551347137290423,0.9846257837040508,0.6730060970271642,0.003080832908103126,0.9411013885380247,0.47059299115851905,0.10939316897201612,0.0020296052284569654,0.2604777726569246,0.003516160658127654


# sql
## create db

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///database1.db')  #in memory: 'sqlite:///:memory:'
for df in pd.read_csv('data.csv',chunksize=1000,iterator=True):
    df.to_sql('table1',engine,if_exists='append')
    (df+.1).to_sql('table2',engine,if_exists='append')

In [3]:
df = pd.read_sql_query('SELECT c2,c8 '
                       'FROM table1 '
                       'WHERE c5>0.9 '
                       'LIMIT 3', engine)
df.head()

,c2,c8
0,0.972368,0.135409
1,0.641813,0.281087
2,0.856883,0.438412


## merge

In [4]:
from sqlalchemy import create_engine
df=pd.read_sql_query("SELECT a.c1,b.c2,a.c3,b.c4 "
                     "FROM table1 a, table2 b "
                     "WHERE ABS(a.c3-b.c4)<0.01 AND "
                     "a.c5 > b.c6 and "
                     "a.c7 < b.c8 "
                     "LIMIT 10000",
                     engine)

In [5]:
print(df.shape)
df.head()https://www.google.com/search?client=ubuntu&channel=fs&q=compare+cinema+ticket+prices&ie=utf-8&oe=utf-8

(4002, 4)


,c1,c2,c3,c4
0,0.389376,0.483499,0.70721,0.704169
1,0.389376,0.128157,0.70721,0.699534
2,0.389376,0.445102,0.70721,0.698638
3,0.389376,0.808863,0.70721,0.709839
4,0.389376,1.077202,0.70721,0.705761


# dask

In [4]:
import dask.array as da
a2 = da.from_array(np.random.random((1000,100)), chunks=200)
print((a2*4).min())
print((a2*4).min().compute())

dask.array<amin-aggregate, shape=(), dtype=float64, chunksize=()>
7.220128436546958e-05


In [6]:
from dask import dataframe as dd
data = dd.read_csv('data.csv', usecols=['c1','c5'])
data

,c1,c5
npartitions=1,,
,float64,float64
,...,...


In [9]:
print(data.c5.count())
print(dd.compute(data.c5.count()))
print((data.c5*data.c1).min().compute())

dd.Scalar<series-..., dtype=int64>
(100,)
0.0011506127139453315


# datatable

In [ ]:
import datatable as dt
print(dt.__version__)

In [ ]:
df = dt.fread("data.csv")
df.to_numpy()
df.to_pandas()

In [ ]:
df.shape
df.names # list of column names
df.stypes # list of column types
df.head(10)

In [ ]:
df.sum()
df.sd()
df.mode()
df.nmodal()
df.nunique()
df.max()
df.min()
df.mean()

In [ ]:
df[:,'column_name']
df[:5,:3]
df.sort('column_name')
df[dt.f.loan_amnt>dt.f.funded_amnt,"loan_amnt"]

In [ ]:
del df[:,'column_name']

In [ ]:
# groupby
for i in range(100):
    df[:, dt.sum(dt.f.funded_amnt), dt.by(dt.f.grade)]
    
## equiv pandas command
for i in range(100):
    pandas_df.groupby("grade")["funded_amnt"].sum()

In [ ]:
df.to_csv('filename.csv')